In [1]:
from keras.models import load_model
model = load_model('whale.flukes.3354_classes.weights.best.hdf5')

Using TensorFlow backend.


# Classify new_whale image

In [8]:
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

target_size = (150, 150)
# new whale
# test_image= image.load_img('train/fe4c0693.jpg', target_size = target_size) 
# 0014cfdf w_f22f3e3
test_image= image.load_img('train/0014cfdf.jpg', target_size = target_size) 

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = preprocess_input(test_image)

result = model.predict(test_image)

print(result[0])
_, sorted_classes = zip(*sorted(zip(result[0], range(10)), reverse=True))
print(sorted_classes)
print(result[0][9])

[0. 0. 0. ... 0. 0. 0.]
(9, 8, 7, 6, 5, 4, 3, 2, 1, 0)
0.0
